# Aggregate the WECC 2032 ADS GridView Load Data by NERC Region


In [2]:
# Start by importing the packages we need:
import os
import glob
import datetime
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Set the Directory Structure

In [7]:
# Identify the data input and image output directory:
ads_load_data_dir =  '/Users/burl878/Documents/Code/code_repos/gdo_climate_toolsuite_ads2032_loads/data/'
ba_mapping_dir =  '/Users/burl878/Documents/Code/code_repos/gdo_climate_toolsuite_visualizations/data/'
data_output_dir = '/Users/burl878/Documents/Code/code_repos/gdo_climate_toolsuite_visualizations/data/nerc_region_load_data/'


## Suppress Future Warnings


In [4]:
warnings.simplefilter(action='ignore', category=FutureWarning)


## Create a Function to Aggregate the 2032 WECC ADS GridView Data by NERC Region


In [15]:
def aggregate_gridview_load_data(start_year: int, end_year: int, ba_mapping_dir: str, ads_load_data_dir: str, data_output_dir: str):

    # Loop over the years of TELL load data:
    for year in range(start_year, end_year, 1):
        # Read in the raw data .csv file for the TELL weather year:
        gv_df = pd.read_csv((ads_load_data_dir + 'TELL_Loads/TELL_Loads_2032_Based_on_' + str(year) + '_Weather.csv'))

        # Subset to just hourly demand by BA, drop the index column, and convert the values to floats:
        gv_df = gv_df[1:8785]
        del gv_df["Index"]
        gv_df = gv_df.astype(np.float64)

        # Strip the unecessary bits from the column names:
        gv_df.columns = gv_df.columns.str.replace("_2032.dat", "")
        gv_df.columns = gv_df.columns.str.replace("Load_", "")

        # Delete the BAs in Canada and Mexico and those with zero loads:
        del gv_df['AESO'], gv_df['BCHA'], gv_df['CFE'], gv_df['TH_Malin'], gv_df['TH_Mead'], gv_df['TH_PV']
        
        # Sum the data across BAs in each NERC region:
        gv_df['WECC'] = gv_df.sum(axis=1).round(2)
        gv_df['WECC: AZ-NM-SNV'] = (gv_df['AZPS'] + gv_df['EPE'] + gv_df['IID'] + gv_df['PNM'] + gv_df['SRP'] 
                                    + gv_df['TEPC'] + gv_df['WALC']).round(2)
        gv_df['WECC: CA-MX'] = (gv_df['BANC'] + gv_df['CIPB'] + gv_df['CIPV'] + gv_df['CISC'] + gv_df['CISD']
                                + gv_df['LDWP'] + gv_df['TIDC'] + gv_df['VEA']).round(2)
        gv_df['WECC: NWPP'] = (gv_df['AVA'] + gv_df['BPAT'] + gv_df['CHPD'] + gv_df['DOPD'] + gv_df['GCPD'] 
                               + gv_df['IPFE'] + gv_df['IPMV'] + gv_df['IPTV'] + gv_df['NEVP'] + gv_df['NWMT'] 
                               + gv_df['PACW'] + gv_df['PAID'] + gv_df['PAUT'] + gv_df['PAWY'] + gv_df['PGE'] 
                               + gv_df['PSEI'] + gv_df['SCL'] + gv_df['SPPC'] + gv_df['TPWR'] + gv_df['WAUW']).round(2)
        gv_df['WECC: RMPA'] = (gv_df['PSCO'] + gv_df['WACM']).round(2)

        # Make a copy of just the WECC data:
        wecc_df = gv_df[['WECC','WECC: AZ-NM-SNV','WECC: CA-MX','WECC: NWPP', 'WECC: RMPA']].copy()

        # Add in columns with the year and hour of the year:
        wecc_df['Year'] = year
        wecc_df['Hour_of_Year'] = range(len(wecc_df))

        # Compute the actual date:
        wecc_df['Time_UTC'] = pd.Timestamp((str(year) + '-01-01')) + pd.to_timedelta(wecc_df.Hour_of_Year, unit='H')

        # Set date parameters as datetime variables:
        wecc_df['Time_UTC'] = pd.to_datetime(wecc_df['Time_UTC'])

        # Only keep the columns we need:
        temp_df = wecc_df[['Time_UTC','WECC','WECC: AZ-NM-SNV','WECC: CA-MX','WECC: NWPP', 'WECC: RMPA']].copy()

        # Aggregate the output into a new dataframe:
        if year == start_year:
           output_df = temp_df
        else:
           output_df = pd.concat([output_df, temp_df])

        # Clean up and move to the next year:
        del gv_df, wecc_df, temp_df

    # Replace zeros with NaN and reset the index value:
    output_df.replace(0, np.nan, inplace=True)
    output_df.reset_index(inplace=True, drop=True)
    
    # Set the output filename:
    output_filename = ('WECC_Hourly_Loads_' + str(start_year) + '_to_' + str(end_year) + '.csv')
        
    # Write out the dataframe to a .csv file:
    output_df.to_csv((os.path.join(data_output_dir, output_filename)), sep=',', index=False)
    
    # Return the output dataframe:
    return output_df


In [9]:
# Execute the function:
output_df = aggregate_gridview_load_data(start_year = 1980,
                                         end_year = 2025, 
                                         ba_mapping_dir = ba_mapping_dir,
                                         ads_load_data_dir = ads_load_data_dir,
                                         data_output_dir = data_output_dir)

output_df


,Time_UTC,WECC,WECC: AZ-NM-SNV,WECC: CA-MX,WECC: NWPP,WECC: RMPA
0,1980-01-01 00:00:00,95949.05,14931.23,33791.59,37148.77,10077.46
1,1980-01-01 01:00:00,100408.36,16187.90,35373.96,38451.96,10394.54
2,1980-01-01 02:00:00,102709.84,16705.44,36291.32,39070.34,10642.74
3,1980-01-01 03:00:00,104593.05,16979.15,37561.18,39582.15,10470.57
4,1980-01-01 04:00:00,104588.55,16854.51,38202.27,39390.51,10141.26
...,...,...,...,...,...,...
395275,2024-12-31 19:00:00,97933.90,13563.27,32934.45,41007.93,10428.25
395276,2024-12-31 20:00:00,97279.37,13433.90,32888.81,40589.88,10366.78
395277,2024-12-31 21:00:00,97107.29,13488.08,33062.90,40220.56,10335.75
395278,2024-12-31 22:00:00,NaN,NaN,NaN,NaN,NaN


## Create a Function to Calculate the Mean, Min, and Max Load by Day


In [5]:
def process_daily_data(start_year: int, end_year: int, data_output_dir: str):

    # Read in data processed using the function created above:
    load_df = pd.read_csv((data_output_dir + 'WECC_Hourly_Loads_' + str(start_year) + '_to_' + str(end_year) + '.csv'))

    # Set the time as datetime variable:
    load_df['Time_UTC'] = pd.to_datetime(load_df['Time_UTC'])

    # Extract the day from the datetime variable:
    load_df['Date'] = load_df['Time_UTC'].dt.date

    # Compute the daily max for each region:
    load_df['WECC_Max'] = load_df.groupby('Date')['WECC'].transform('max').round(2)
    load_df['WECC: AZ-NM-SNV_Max'] = load_df.groupby('Date')['WECC: AZ-NM-SNV'].transform('max').round(2)
    load_df['WECC: CA-MX_Max'] = load_df.groupby('Date')['WECC: CA-MX'].transform('max').round(2)
    load_df['WECC: NWPP_Max'] = load_df.groupby('Date')['WECC: NWPP'].transform('max').round(2)
    load_df['WECC: RMPA_Max'] = load_df.groupby('Date')['WECC: RMPA'].transform('max').round(2)
        
    # Subset and reorder the columns, drop duplicates, and sort chronologically:
    output_df = load_df[['Date', 'WECC_Max', 'WECC: AZ-NM-SNV_Max', 'WECC: CA-MX_Max', 'WECC: NWPP_Max', 'WECC: RMPA_Max']]
    output_df = output_df.drop_duplicates()
    output_df = output_df.sort_values('Date')

    # Rename the columns for simplicity:
    output_df.rename(columns={'WECC_Max': 'WECC', 'WECC: AZ-NM-SNV_Max': 'WECC: AZ-NM-SNV', 'WECC: CA-MX_Max': 'WECC: CA-MX',
                              'WECC: NWPP_Max': 'WECC: NWPP', 'WECC: RMPA_Max': 'WECC: RMPA'}, inplace=True)

    # Replace zeros with NaN and reset the index value:
    output_df.replace(0, np.nan, inplace=True)
    output_df.reset_index(inplace=True, drop=True)
    
    # Set the output filename:
    output_filename = ('WECC_Daily_Peak_' + str(start_year) + '_to_' + str(end_year) + '.csv')
        
    # Write out the dataframe to a .csv file:
    output_df.to_csv((os.path.join(data_output_dir, output_filename)), sep=',', index=False)
    
    return output_df
    

In [6]:
# Execute the function:
output_df = process_daily_data(start_year = 1980,
                               end_year = 2025, 
                               data_output_dir = data_output_dir)

output_df


,Date,WECC,WECC: AZ-NM-SNV,WECC: CA-MX,WECC: NWPP,WECC: RMPA
0,1980-01-01,104593.05,16979.15,38202.27,39582.15,10642.74
1,1980-01-02,108800.98,17896.96,39919.05,41833.58,10943.41
2,1980-01-03,110798.56,18590.14,40338.35,42876.67,11353.22
3,1980-01-04,112635.19,18598.77,40761.39,43860.90,11219.94
4,1980-01-05,107661.54,17008.60,38413.88,42244.33,10838.70
...,...,...,...,...,...,...
16432,2024-12-27,106513.83,16504.04,38190.69,41148.10,10804.61
16433,2024-12-28,102396.05,15867.29,36188.85,39841.89,10528.74
16434,2024-12-29,101220.76,15631.52,36083.97,39231.59,10273.68
16435,2024-12-30,106345.69,16300.64,37988.52,41630.20,10502.86


## Process the Base WECC 2032 ADS Data


In [26]:
# Read in the raw data .csv file for the TELL weather year:
gv_df = pd.read_csv((ads_load_data_dir + '2032ADS_EEI_Format_Data.csv'))

# Subset to just hourly demand by BA, drop the index column, and convert the values to floats:
gv_df = gv_df[1:8761]
del gv_df["Index"]
gv_df = gv_df.astype(np.float64)

# Strip the unecessary bits from the column names:
gv_df.columns = gv_df.columns.str.replace("_2032.dat", "")
gv_df.columns = gv_df.columns.str.replace("Load_", "")

# Delete the BAs in Canada and Mexico and those with zero loads:
del gv_df['AESO'], gv_df['BCHA'], gv_df['CFE'], gv_df['TH_Malin'], gv_df['TH_Mead'], gv_df['TH_PV']
        
# Sum the data across BAs in each NERC region:
gv_df['WECC'] = gv_df.sum(axis=1).round(2)
gv_df['WECC: AZ-NM-SNV'] = (gv_df['AZPS'] + gv_df['EPE'] + gv_df['IID'] + gv_df['PNM'] + gv_df['SRP'] 
                            + gv_df['TEPC'] + gv_df['WALC']).round(2)
gv_df['WECC: CA-MX'] = (gv_df['BANC'] + gv_df['CIPB'] + gv_df['CIPV'] + gv_df['CISC'] + gv_df['CISD']
                        + gv_df['LDWP'] + gv_df['TIDC'] + gv_df['VEA']).round(2)
gv_df['WECC: NWPP'] = (gv_df['AVA'] + gv_df['BPAT'] + gv_df['CHPD'] + gv_df['DOPD'] + gv_df['GCPD'] 
                       + gv_df['IPFE'] + gv_df['IPMV'] + gv_df['IPTV'] + gv_df['NEVP'] + gv_df['NWMT'] 
                       + gv_df['PACW'] + gv_df['PAID'] + gv_df['PAUT'] + gv_df['PAWY'] + gv_df['PGE'] 
                       + gv_df['PSEI'] + gv_df['SCL'] + gv_df['SPPC'] + gv_df['TPWR'] + gv_df['WAUW']).round(2)
gv_df['WECC: RMPA'] = (gv_df['PSCO'] + gv_df['WACM']).round(2)

# Make a copy of just the WECC data:
wecc_df = gv_df[['WECC','WECC: AZ-NM-SNV','WECC: CA-MX','WECC: NWPP', 'WECC: RMPA']].copy()

# Add in columns with the year and hour of the year:
wecc_df['Year'] = 2018
wecc_df['Hour_of_Year'] = range(len(wecc_df))

# Compute the actual date:
wecc_df['Time_UTC'] = pd.Timestamp((str(2018) + '-01-01')) + pd.to_timedelta(wecc_df.Hour_of_Year, unit='H')

# Set date parameters as datetime variables:
wecc_df['Time_UTC'] = pd.to_datetime(wecc_df['Time_UTC'])

# Only keep the columns we need:
output_df = wecc_df[['Time_UTC','WECC','WECC: AZ-NM-SNV','WECC: CA-MX','WECC: NWPP', 'WECC: RMPA']].copy()

# Replace zeros with NaN and reset the index value:
output_df.replace(0, np.nan, inplace=True)
output_df.reset_index(inplace=True, drop=True)
    
# Set the output filename:
output_filename = ('WECC_ADS_Hourly_Loads_2018.csv')
        
# Write out the dataframe to a .csv file:
output_df.to_csv((os.path.join(data_output_dir, output_filename)), sep=',', index=False)


In [27]:
output_df


,Time_UTC,WECC,WECC: AZ-NM-SNV,WECC: CA-MX,WECC: NWPP,WECC: RMPA
0,2018-01-01 00:00:00,87681.52,14055.07,29002.46,36027.0,8597.0
1,2018-01-01 01:00:00,83594.20,13697.07,26836.14,34665.0,8396.0
2,2018-01-01 02:00:00,81414.37,13535.07,25329.30,34206.0,8344.0
3,2018-01-01 03:00:00,80345.18,13570.07,24311.11,34070.0,8394.0
4,2018-01-01 04:00:00,81503.98,13860.07,24004.91,34966.0,8673.0
...,...,...,...,...,...,...
8755,2018-12-31 19:00:00,114824.89,17948.17,41836.72,43466.0,11574.0
8756,2018-12-31 20:00:00,112389.74,17747.17,41032.57,42396.0,11214.0
8757,2018-12-31 21:00:00,108848.42,17226.17,39870.25,40964.0,10788.0
8758,2018-12-31 22:00:00,103374.22,16363.17,37856.04,39062.0,10093.0
